In [9]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

In [10]:
import cv2
import math

#求灰度共生矩阵
def getGlcm(input, d_x = 0, d_y = 1, gray_level = 16):
    srcdata=input.copy()
    ret = np.zeros([1,gray_level, gray_level])
    (height,width) = input.shape[1], input.shape[2]
    
    max_gray_level = input.max()
    
    #若灰度级数大于gray_level，则将图像的灰度级缩小至gray_level，减小灰度共生矩阵的大小
    if max_gray_level > gray_level:
        for j in range(height):
            for i in range(width):
                srcdata[:,j:j+1,i:i+1] = int(srcdata[:,j:j+1,i:i+1])*gray_level / max_gray_level

    for j in range(height - d_y):
        for i in range(width - d_x):
            rows = int(srcdata[:,j:j+1,i:i+1])
            cols = int(srcdata[:,j + d_y: j + d_y + 1, i + d_x : i + d_x + 1])
            ret[:, rows: rows + 1, cols : cols+1] = ret[:,rows:rows+1,cols:cols+1] + 1

    for i in range(gray_level):
        for j in range(gray_level):
            ret[:, i : i+1, j: j+1]/=float(height * width)

    return ret

#每一张图的灰度共生矩阵对应的特征（输出为4个值）
def feature_extract(p,gray_level = 16):
    Con = 0.0
    Eng = 0.0
    Asm = 0.0
    Idm = 0.0
    for i in range(gray_level):
        for j in range(gray_level):
            Con += (i - j) * (i - j) * p[:, i : i+1, j : j+1]
            Asm += p[:, i : i+1, j : j+1] * p[:, i : i+1, j : j+1]
            Idm += p[:, i : i+1, j : j+1] / (1 + (i - j) * (i - j))
            if p[ : , i : i+1, j : j+1] > 0:
                Eng = Eng + p[:, i : i+1, j : j+1] * math.log(p[:, i : i+1, j : j+1])
                
    return Asm, Con, -Eng, Idm

def getGlcm_whl(img, patchsize = np.array([16,16])):
    p_1 = patchsize[0]
    p_2 = patchsize[1]
    if len(img.shape) != 3:
        print("The img must be a tuple ")
    else:
        is_1 = img.shape[1]
        is_2 = img.shape[2]
        if (is_1 % p_1 != 0) or (is_2 % p_2 != 0):
            print("The number of rows and cols must be multiple of patchsize")
        else:
            row_num = int(is_1 / p_1)
            col_num = int(is_2 / p_2)
            ASM = np.zeros([row_num, col_num])
            CON = np.zeros([row_num, col_num])
            ENG = np.zeros([row_num, col_num])
            IDM = np.zeros([row_num, col_num])
            for t in range(row_num):
                for k in range(col_num):
                    img_pt = img[:, t * p_1 : (t + 1) * p_1, k * p_2 : (k + 1) * p_2]
                    img_glcm = getGlcm(img_pt)
                    img_ft = feature_extract(img_glcm)
                    img_ft = np.asarray(img_ft)
                    ASM[t : t + 1, k : k + 1] = img_ft[0]
                    CON[t : t + 1, k : k + 1] = img_ft[1]
                    ENG[t : t + 1, k : k + 1] = img_ft[2]
                    IDM[t : t + 1, k : k + 1] = img_ft[3]
    return ASM.flatten(), CON.flatten(), ENG.flatten(), IDM.flatten()

In [8]:
X_tumor = np.load("CT_tumoronly.npy")
patient_location = np.load("patient_location.npy")
number_patient = patient_location.shape[0]-1

In [33]:
#data patient_location.shape:(108,1)
def Glcm_PCA(X,number_patient,patient_location,patchsize = np.array([32,32]),dim = 1):
    pca = PCA(n_components = dim)
    patient_location = patient_location.astype('int')
    result = np.zeros([number_patient,16])
    for i in range(number_patient):
        lenth = patient_location[i+1]-patient_location[i]
        gray_matrix = np.zeros([4,lenth*1024])
        for j in range(lenth):
            print(j)
            gray_matrix[0,j*1024:j*1024+1024],gray_matrix[1,j*1024:j*1024+1024],gray_matrix[2,j*1024:j*1024+1024],gray_matrix[3,j*1024:(j+1)*1024] = getGlcm_whl(X[patient_location[i]+j].reshape(1,512,512))
        gray_matrix = gray_matrix.reshape(16,256*lenth)
        gray_matrix_new=pca.fit_transform(gray_matrix)
        result[i,:] = gray_matrix_new.flatten()
    return result
            
        

In [ ]:
result = Glcm_PCA(X_input,number_patient,patient_location)

In [71]:
np.save("feature_two.npy",result)

In [65]:
#所有人的所有掩膜图像
mask_all = np.load("C:/Users/Ye Xiaoqing/Desktop/B/model_1_data/mask_all_512.npy")
index = np.unique(np.where(mask_all == 1)[0])#调出所有人有肿瘤的掩膜图像的位置
index_size = index.shape[0]
number_patient = 20#病人人数
patient_mask_number = np.zeros(number_patient+10)#每个病人有肿瘤的掩膜图像的张数
i = 0
for k in range(index_size-1):
    print(i)
    if (index[k+1]-index[k])>3:
        i = i+1
    else:
        patient_mask_number[i] = patient_mask_number[i]+1
patient_location = np.zeros(number_patient+1)
for j in range(1,number_patient+1):
    patient_location[j] = patient_mask_number[0:j].sum()#每个病人的肿瘤掩膜图像在所有人的有肿瘤的掩膜图像构成的队列中的起始位置

0
1
1
1
1
2
2
2
2
2
2
2
2
3
4
4
4
5
5
5
5
5
5
5
5
6
6
7
7
7
7
7
7
7
7
7
8
8
9
9
9
10
10
10
10
10
11
11
11
11
11
11
12
12
13
13
13
13
13
14
14
14
14
14
14
14
14
15
16
16
16
16
16
16
16
16
16
16
17
17
18
18
18
19
19
19
19
19
19
20
21
21
21
21
21
21
21
21
21
21
21
21


In [89]:
patient_index = np.load("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/index_patient.npy")

In [90]:
patient_mask_number = np.zeros(number_patient)
for i in range(number_patient):
    patient_mask_number[i] = np.where(patient_index == (i+1))[0].shape[0]
patient_location = np.zeros(number_patient+1)
for j in range(1,number_patient+1):
    patient_location[j] = patient_mask_number[0:j].sum()

In [91]:
X_input = np.load("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/tumor_only.npy")
f2_model1 = Glcm_PCA(X_input, number_patient, patient_location)

0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
0
1
2
3
4
5
6
7
8
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
10
11
12
13
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
0
1
2
0
1
2
3
4
5
6
7
0
1
2


In [93]:
np.save("C:/Users/Ye Xiaoqing/Desktop/B/model_2_data/f2_model2.npy",f2_model1 )

In [94]:
f2_model1.shape

(20, 16)